In [ ]:
import pandas as pd
df = pd.read_csv("lexibank_meta.csv")
#df = pd.read_csv("SequenceComparison_meta.csv")
sub_df = df[
           (df["forms.csv"] == 1) & 
           (df["cognates.csv"] == 1) 
           #(df["values.csv"] == 1)
            ] 
           # 
for name in sub_df["name"]:
    print(name)

In [ ]:
#old code, paths not up to date, just to trace back what I have done


import os
import json
import shutil

repos_phlorest = ['birchallchapacuran', 'gerarditupi', 'dyenindoeuropean', 'utoaztecan', 
                  'grollemundbantu', 'iecor', 'kitchensemitic', 'dravlex', 'leekoreanic',
                 'leejaponic', 'leeainu', 'nagarajakhasian', 'powerma', 'robinsonap', 'sagartst',
                 'savelyevturkic', 'walkerarawakan']
done_repos = set(repos_phlorest)
d = "../done/lexibankMrbayes/"
with os.scandir(d) as it:
    for entry in it:
        if not entry.is_file():
            continue
        done_repos.add(entry.name.split('.')[0])

print(done_repos)
content = open("lexibank_repos.json", "r").read()
#content = open("SequenceComparison_repos.json", "r").read()
repo_json = json.loads(content)
available_files = {}
for repo in repo_json:
    name = repo["name"]
    print(name)
    url = repo["html_url"]
    #try:
    #    os.system("git clone " + url + ".git")
    #except:
    #    "Cloning failed somehow..."
    d = os.path.join(name, "cldf")
    files = []
    if not os.path.isdir(d):
        available_files[name] = files
        continue
    with os.scandir(d) as it:
        for entry in it:
            if not entry.is_file():
                continue
            if entry.name.endswith(".csv"):
                files.append(entry.name)
    #shutil.rmtree(name)
    files.sort()
    available_files[name] = files
    
    
    
all_file_names = set()
for (name, files) in available_files.items():
    all_file_names = all_file_names.union(set(files))
all_file_names = list(all_file_names)
all_file_names.sort()

matrix = {}
for (name, files) in available_files.items():
    bits =  ['0' for _ in range(len(all_file_names) + 1)]
    for (i, file_name) in enumerate(all_file_names):
        if file_name in files:
            bits[i] = '1'
    if name in done_repos:
        bits[-1] = '1'
    matrix[name] = bits
header = ['name'] + all_file_names + ['done']
matrix = [header] + [[name] + bits for (name, bits) in matrix.items()]
out_file = open("lexibank_meta.csv", "w+")
#out_file = open("SequenceComparison_meta.csv", "w+")
for row in matrix:
    out_file.write(','.join(row) + '\n')